<a href="https://colab.research.google.com/github/Sopulu-max/Exploring_Python/blob/main/GameOfLife.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import sys, argparse
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [10]:
ON = 255
OFF = 0
vals = [ON, OFF]

def random_grid(N):
    """returns a grid of N*N randomValues"""
    return np.random.choice(vals, N * N, p=[0.2, 0.8]).reshape(N, N)


def add_glider(i, j, grid):
    """adds a glider with top left cell at (i, j)"""
    glider = np.array([[0, 0, 255], [255, 0, 255], [0, 255, 255]])
    grid[i:i + 3, j:j + 3] = glider

In [11]:
def update(frame_num, img, grid, N):
    # copy grid since we require 8 neighbours for calculation
    # and we go line by line
    new_grid = grid.copy()
    for i in range(N):
        for j in range(N):
            # compute 8-neighbour sum using toroidal boundary condions
            # x and y wrap around so that the simulation takes place on a toroidal surface
            total = int((grid[i, (j - 1) % N] + 
                         grid[i, (j + 1) % N] + 
                         grid[(i - 1) % N, j] +
                         grid[(i + 1) % N, j] + 
                         grid[(i - 1) % N, (j - 1) % N] + 
                         grid[(i - 1) % N, (j + 1) % N] +
                         grid[(i + 1) % N, (j - 1) % N] + 
                         grid[(i + 1) % N, (j + 1) % N]) / 255)
            # apply conway's rules
            if grid[i, j] == ON:
                if (total < 2) or (total > 3):
                    new_grid[i, j] = OFF
            else:
                if total == 3:
                    new_grid[i, j] = ON

    # update data
    img.set_data(new_grid)
    grid[:] = new_grid[:]
    return img

In [12]:
def main():
    parser = argparse.ArgumentParser(
        description="Runs Conway's Game Of Life Simulation")
    parser.add_argument('--grid-size', dest='N', required=False)
    parser.add_argument('--mov-file', dest='movfile', required=False)
    parser.add_argument('--interval', dest='interval', required=False)
    parser.add_argument('--glider', action='store_true', required=False)
    parser.add_argument('--gosper', action='store_true', required=False)
    args = parser.parse_args()

    # set grid size
    N = 100
    if args.N and int(args.N) > 8:
        N = int(args.N)

    # set animation update interval
    update_interval = 50
    if args.interval:
        update_interval = int(args.interval)

    # declare grid
    grid = np.array([])
    # check if glider demo flag is specified
    if args.glider:
        grid = np.zeros(N * N).reshape(N, N)
        add_glider(1, 1, grid)
    else:
        # populate grid with random on/off - more off than on
        grid = random_grid(N)

    # set up the animation
    fig, ax = plt.subplots()
    img = ax.imshow(grid, interpolation='nearest')
    ani = animation.FuncAnimation(fig,
                                  update,
                                  fargs=(
                                      img,
                                      grid,
                                      N,
                                  ),
                                  frames=10,
                                  interval=update_interval,
                                  save_count=50)
    # number of frames
    # set the output file
    if args.movfile:
        ani.save(args.movfile, fps=30, extra_args=['-vcodec', 'libx264'])

    plt.show()